# Streaming Data Warehousing Demo

![alt text](images/delta_dwh.png "Data Warehouse")

### Bronze layer (raw data)
The Bronze layer is where we land all the data from external source systems. The table structures in this layer correspond to the source system table structures "as-is," along with any additional metadata columns that capture the load date/time, process ID, etc. The focus in this layer is quick Change Data Capture and the ability to provide an historical archive of source (cold storage), data lineage, auditability, reprocessing if needed without rereading the data from the source system.

### Silver layer (cleansed and conformed data)
In the Silver layer of the lakehouse, the data from the Bronze layer is matched, merged, conformed and cleansed ("just-enough") so that the Silver layer can provide an "Enterprise view" of all its key business entities, concepts and transactions. (e.g. master customers, stores, non-duplicated transactions and cross-reference tables).

The Silver layer brings the data from different sources into an Enterprise view and enables self-service analytics for ad-hoc reporting, advanced analytics and ML. It serves as a source for Departmental Analysts, Data Engineers and Data Scientists to further create projects and analysis to answer business problems via enterprise and departmental data projects in the Gold Layer.

In the lakehouse data engineering paradigm, typically the ELT methodology is followed vs. ETL - which means only minimal or "just-enough" transformations and data cleansing rules are applied while loading the Silver layer. Speed and agility to ingest and deliver the data in the data lake is prioritized, and a lot of project-specific complex transformations and business rules are applied while loading the data from the Silver to Gold layer. From a data modeling perspective, the Silver Layer has more 3rd-Normal Form like data models. Data Vault-like, write-performant data models can be used in this layer.

### Gold layer (curated business-level tables)
Data in the Gold layer of the lakehouse is typically organized in consumption-ready "project-specific" databases. The Gold layer is for reporting and uses more de-normalized and read-optimized data models with fewer joins. The final layer of data transformations and data quality rules are applied here. Final presentation layer of projects such as Customer Analytics, Product Quality Analytics, Inventory Analytics, Customer Segmentation, Product Recommendations, Marking/Sales Analytics etc. fit in this layer. We see a lot of Kimball style star schema-based data models or Inmon style Data marts fit in this Gold Layer of the lakehouse.

So you can see that the data is curated as it moves through the different layers of a lakehouse. In some cases, we also see that lot of Data Marts and EDWs from the traditional RDBMS technology stack are ingested into the lakehouse, so that for the first time Enterprises can do "pan-EDW" advanced analytics and ML - which was just not possible or too cost prohibitive to do on a traditional stack. (e.g. IoT/Manufacturing data is tied with Sales and Marketing data for defect analysis or health care genomics, EMR/HL7 clinical data markets are tied with financial claims data to create a Healthcare Data Lake for timely and improved patient care analytics.)

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder.appName("JAMBA_JUICE") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()

# Bronze Layer - Data Ingestion
The Bronze layer is where we land all the data from external source systems. The table structures in this layer correspond to the source system table structures "as-is," along with any additional metadata columns that capture the load date/time, process ID, etc. The focus in this layer is quick Change Data Capture and the ability to provide an historical archive of source (cold storage), data lineage, auditability, reprocessing if needed without rereading the data from the source system.

The following meta columns are added to the table:
- **meta_created**: Timestamp from when the row was ingested
- **meta_filename**: Filename from which the row was ingested

In [4]:
from pyspark.sql.functions import current_timestamp, input_file_name

# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

def bronze_streaming_table(source_folder, target_table):

    # Generates a source path based on table name, reads all files from that and inserts into bronze schema
    query = (
        spark.readStream
        .format("json")
        .load(source_folder)
        .withColumn("meta_created", current_timestamp())
        .withColumn("meta_filename", input_file_name())
        .writeStream
        .outputMode("append")
        .format("delta")
        .trigger(processingTime='10 seconds')
        .option("checkpointLocation", f"spark-warehouse/_checkpoints/{target_table}")
        .queryName(target_table)
        .toTable(target_table)
    )
    return query

bronze_streaming_table(source_folder="data/inventory", target_table="bronze_inventory")
bronze_streaming_table(source_folder="data/product", target_table="bronze_product")
bronze_streaming_table(source_folder="data/sales", target_table="bronze_sales")
bronze_streaming_table(source_folder="data/customer", target_table="bronze_customer")


# Silver Layer - Slowly Changing Dimensions
Type 1 Slowly Changing Dimension: This method overwrites the existing value with the new value and does not retain history. Type 2 Slowly Changing Dimension: This method adds a new row for the new value and maintains the existing row for historical and reporting purposes.

Depending on the slowly changing dimension (SCD) type, the following meta columns will be created:

### SCD Type 1:
- **meta_hashdiff**: Hash key of all non-meta columns. [Read more here.](https://www.tpximpact.com/knowledge-hub/blogs/tech/hash-keys-data-warehousing-2/)
- **meta_last_updated**: Timestamp of when the row was last updated/overwritten.
- **meta_sequence**: Sequence number used for inserting data with duplicates of main key.

### SCD Type 2:
- **meta_hashdiff**: Hash key of all non-meta columns.
- **meta_is_current**: Boolean of whether this row is the current (most recent).
- **meta_valid_from**: Timestamp of when the row was first ingested.
- **meta_valid_to**: Timestamp of then the row was outdated by a newer version (Null if row is current).
- **meta_sequence**: Sequence number used for inserting data with duplicates of main key.

In [5]:
def silver_table_schema(
        table_name : str, 
        surrogate_key : str, 
        source_table : str, 
        scd_type : int
    ):

    # Define table name and surrogate key
    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({surrogate_key} string,"
    
    # Get schema of source table
    source_schema = spark.sql(f"describe table {source_table}").collect()
    for row in source_schema:
        query += f" {row['col_name']} {row['data_type']},"

    # Add extra meta columns depending on SCD (slowly changing dimension) type
    if scd_type == 1:
        query += "meta_hashdiff string, meta_last_updated timestamp, meta_sequence int) USING DELTA"
    elif scd_type == 2:
        query += "meta_hashdiff string, meta_is_current boolean, meta_valid_from timestamp, meta_valid_to timestamp, meta_sequence int) USING DELTA"

    # Run and print SQL query
    spark.sql(query)
    print(query)

silver_table_schema(table_name="silver_sales", surrogate_key="transaction_sid", source_table="bronze_sales", scd_type=1)
silver_table_schema(table_name="silver_inventory", surrogate_key="inventory_sid", source_table="bronze_inventory", scd_type=1)
silver_table_schema(table_name="silver_product", surrogate_key="product_sid", source_table="bronze_product", scd_type=2)
silver_table_schema(table_name="silver_customer", surrogate_key="customer_sid", source_table="bronze_customer", scd_type=2)


CREATE TABLE IF NOT EXISTS silver_sales (transaction_sid string, customer_id bigint, member_discount double, price double, product_id string, quantity bigint, supplement_price double, total_purchase double, transaction_id string, transaction_time string, meta_created timestamp, meta_filename string,meta_hashdiff string, meta_last_updated timestamp, meta_sequence int) USING DELTA
CREATE TABLE IF NOT EXISTS silver_inventory (inventory_sid string, event_time string, existing_level bigint, new_level bigint, product_id string, stock_quantity bigint, meta_created timestamp, meta_filename string,meta_hashdiff string, meta_last_updated timestamp, meta_sequence int) USING DELTA
CREATE TABLE IF NOT EXISTS silver_product (product_sid string, category string, cogs double, contains_caffeine boolean, contains_fruit boolean, contains_nuts boolean, contains_veggies boolean, event_time string, item string, price double, product_id string, size string, meta_created timestamp, meta_filename string,meta_h

# Slowly Changing Dimensions (SCD) Type 1 & 2

In the Type 1 SCD, you simply overwrite data in dimensions. [Read more here.](https://www.sqlshack.com/implementing-slowly-changing-dimensions-scds-in-data-warehouses/)

In [6]:
from pyspark.sql.functions import md5, concat_ws, lit, row_number, column
from pyspark.sql.types import BooleanType, TimestampType, BinaryType
from pyspark.sql.window import Window

def upsert_to_scd_table(
        target_table: str,
        timestamp_key: str,
        join_key: str,
        scd_type: int
    ):

    def _inner_func(df, batch_id):
        
        # Set default values for meta columns depending on SCD type
        if scd_type == 1:
            df  = df.withColumn("meta_last_updated", current_timestamp())
        elif scd_type == 2:
            df = df.withColumn("meta_is_current", lit(1).cast(BooleanType()))
            df = df.withColumn("meta_valid_from", df[timestamp_key])
            df = df.withColumn("meta_valid_to", lit('9999-12-31').cast(TimestampType()))
        else:
            raise ValueError("Currently only supports SCD type 1 and 2")
        
        # Calculate hashdiff
        df = df.withColumn("meta_hashdiff", md5(concat_ws("||", *[c for c in df.columns if "meta_" not in c])))

        # Calculate sequence number
        df = df.withColumn("meta_sequence", row_number().over(Window.partitionBy(join_key).orderBy(timestamp_key)))

        # Reorder columns to match target table (only neccessary for SCD type 2)
        if scd_type == 2:
            df_target = spark.read.table(target_table).limit(1)
            df = df.select(df_target.columns)

        # Create view with source data
        df.createOrReplaceTempView("tempView")

        # Get list of sequences
        lst_sequence = sorted([p.meta_sequence for p in df.select('meta_sequence').distinct().collect()])

        # Loop over the sequence and do merge into statements for either SCD type 1 or 2
        for seq_num in lst_sequence:
            if scd_type == 1:
                query = f"""
                    MERGE INTO {target_table} AS t
                    USING (
                        SELECT *
                        FROM tempView
                        WHERE meta_sequence = {seq_num}
                    ) AS s ON t.{join_key} = s.{join_key}
                    WHEN MATCHED AND t.meta_hashdiff <> s.meta_hashdiff 
                        THEN UPDATE SET *
                    WHEN NOT MATCHED 
                        THEN INSERT *
                """
                df.sparkSession.sql(query)

            elif scd_type == 2:
                merge_query = f"""
                    MERGE INTO {target_table} AS t
                    USING (
                        SELECT * 
                        FROM tempView
                        WHERE meta_sequence = {seq_num}
                    ) AS s ON t.{join_key} = s.{join_key}
                    WHEN MATCHED AND t.meta_is_current = true AND t.meta_hashdiff <> s.meta_hashdiff
                        THEN UPDATE SET meta_is_current = false, meta_valid_to = s.{timestamp_key}
                    WHEN NOT MATCHED 
                        THEN INSERT *
                """
                df.sparkSession.sql(merge_query)

                insert_query = f"""
                    INSERT INTO {target_table}
                    SELECT s.*
                    FROM tempView s
                    JOIN {target_table} t ON t.{join_key} = s.{join_key}
                    WHERE s.meta_sequence = {seq_num}
                    AND t.meta_hashdiff <> s.meta_hashdiff
                """
                df.sparkSession.sql(insert_query)
    
    return _inner_func

In [7]:
def silver_streaming_table(
    source_table : str, 
    target_table : str,
    timestamp_key : str,
    join_key: str,
    surrogate_key : str,
    scd_type: int
):
    # Generates a source path based on table name, reads all files from that and inserts into bronze schema
    query = (
        spark.readStream
        .table(source_table)
        .withColumn(surrogate_key, md5(column(join_key).cast(BinaryType())))
        .writeStream
        .format("delta")
        .foreachBatch(upsert_to_scd_table(
            target_table=target_table, 
            timestamp_key=timestamp_key, 
            join_key=join_key, 
            scd_type=scd_type
        ))
        .outputMode("update")
        .queryName(target_table)
        .start()
    )
    return query

In [8]:
# Create silver tables with SCD type 1
silver_streaming_table (
    source_table="bronze_sales",
    target_table="silver_sales",
    timestamp_key="transaction_time",
    join_key="transaction_id",
    surrogate_key="transaction_sid",
    scd_type=1
)

silver_streaming_table (
    source_table="bronze_inventory",
    target_table="silver_inventory",
    timestamp_key="event_time",
    join_key="event_time",
    surrogate_key="inventory_sid",
    scd_type=1
)

# Create silver tables with SCD type 2
silver_streaming_table(
    source_table = "bronze_product",
    target_table = "silver_product",
    join_key = "product_id",
    timestamp_key = "event_time",
    surrogate_key = "product_sid",
    scd_type=2
)

silver_streaming_table(
    source_table = "bronze_customer",
    target_table = "silver_customer",
    join_key = "customer_id",
    timestamp_key = "event_time",
    surrogate_key = "customer_sid",
    scd_type=2
)

# Gold Layer - Facts and Dimensions
## What is Dimensional Modeling
The data model used to store data in the denormalized form is called Dimensional Modeling. It is the technique of storing data in a Data Warehouse in such a way that enables fast query performance and easy access to its business users. It involves creating a set of dimensional tables that are designed to support business intelligence and reporting needs.

The core concept of dimensional modeling is the creation of a star schema. It is called so as the tables are arranged in the form of a star.

![alt text](images/star_schema.png "Star Schema")

Dimensional modeling includes facts and dimensions. Let’s have a basic idea of what Facts and Dimensions are.

## Fact Tables
Fact tables are the heart of a data warehouse. They contain quantitative data, often referred to as measures or metrics, and are the focus of most data analysis. These tables store data related to business transactions and events, such as sales figures, revenue, or quantities sold. In essence, fact tables provide the “what” in data analysis.

## Dimension Tables
Dimension tables, on the other hand, offer context to the data stored in fact tables. They provide descriptive information that helps users understand the “who,” “where,” and “when” aspects of the data.

In [9]:
def create_gold_table_schema(
        table_name : str, 
        surrogate_key : str, 
        source_table : str, 
        dim_table_refs : dict,
        include_unknown_column : bool
    ):

    # Define table name and surrogate key
    query = f"CREATE OR REPLACE TABLE {table_name} ({surrogate_key} string"

    # Loop through and add surrogate keys for foreign keys
    for row in dim_table_refs:
        query += f", {row['surrogate_key']} string"

    # Get schema of source table
    source_schema = spark.sql(f"describe table {source_table}").collect()
    for row in source_schema:
        if row['col_name'] != surrogate_key:
            query += f", {row['col_name']} {row['data_type']}"

    query += ") USING DELTA;"

    print(query)
    spark.sql(query)

    if include_unknown_column:
        print(f"INSERT INTO {table_name} ({surrogate_key}) VALUES ('N/A')")
        spark.sql(f"INSERT INTO {table_name} ({surrogate_key}) VALUES ('N/A')")


In [10]:
create_gold_table_schema (
    table_name="gold_dim_product",
    source_table="silver_product",
    surrogate_key="product_sid",
    dim_table_refs=[],
    include_unknown_column=True
)

create_gold_table_schema (
    table_name="gold_dim_customer",
    source_table="silver_customer",
    surrogate_key="customer_sid",
    dim_table_refs=[],
    include_unknown_column=True
)

create_gold_table_schema (
    table_name="gold_fact_sales",
    source_table="silver_sales",
    surrogate_key="transaction_sid",
    dim_table_refs=[
        {"table_name": "gold_dim_product", "join_key": "product_id", "surrogate_key": "product_sid"},
        {"table_name": "gold_dim_customer", "join_key": "customer_id", "surrogate_key": "customer_sid"},
    ],
    include_unknown_column=False
)

create_gold_table_schema (
    table_name="gold_fact_inventory",
    source_table="silver_inventory",
    surrogate_key="inventory_sid",
    dim_table_refs=[{"table_name": "gold_dim_product", "join_key": "product_id", "surrogate_key": "product_sid"}],
    include_unknown_column=False
)

CREATE OR REPLACE TABLE gold_dim_product (product_sid string, category string, cogs double, contains_caffeine boolean, contains_fruit boolean, contains_nuts boolean, contains_veggies boolean, event_time string, item string, price double, product_id string, size string, meta_created timestamp, meta_filename string, meta_hashdiff string, meta_is_current boolean, meta_valid_from timestamp, meta_valid_to timestamp, meta_sequence int) USING DELTA;
INSERT INTO gold_dim_product (product_sid) VALUES ('N/A')
CREATE OR REPLACE TABLE gold_dim_customer (customer_sid string, address string, credit_card_expire string, credit_card_number string, customer_id bigint, email string, event_time string, full_name string, phone_number string, meta_created timestamp, meta_filename string, meta_hashdiff string, meta_is_current boolean, meta_valid_from timestamp, meta_valid_to timestamp, meta_sequence int) USING DELTA;
INSERT INTO gold_dim_customer (customer_sid) VALUES ('N/A')
CREATE OR REPLACE TABLE gold_fac

In [16]:
from spark_func_utils import generate_dim_table_references

def process_gold_table(
    source_table : str, 
    target_table : str,
    surrogate_key : str,
    delta_load_column : str = None,
    timestamp_key : str = "",
    dim_table_refs : list[dict] = []
):
    print("\nProcessesing changes into", target_table)

    # Generate and run SQL query
    if "fact" in target_table:
        query = generate_dim_table_references(
            source_table=source_table,
            timestamp_key=timestamp_key, 
            dim_table_refs=dim_table_refs)
    else:
        query = f"SELECT s.* FROM {source_table} s"

    # Add delta load logic if the target_table table already exists
    if delta_load_column:
        query += f"\n WHERE s.{delta_load_column} > (SELECT COALESCE(MAX({delta_load_column}), '1970-01-01') FROM {target_table})"

    # Run query and create view
    spark.sql(query).createOrReplaceTempView("tempView")

    # Merge into target table 
    merge_query = f"""
        MERGE INTO {target_table} AS t
        USING tempView AS s
            ON t.{surrogate_key} = s.{surrogate_key}
        WHEN MATCHED AND t.meta_hashdiff <> s.meta_hashdiff 
            THEN UPDATE SET *
        WHEN NOT MATCHED 
            THEN INSERT *
    """
    spark.sql(merge_query).show()
    

In [17]:
process_gold_table (
    source_table="silver_product",
    target_table="gold_dim_product",
    surrogate_key="product_sid",
    delta_load_column="event_time"
)

process_gold_table (
    source_table="silver_customer",
    target_table="gold_dim_customer",
    surrogate_key="customer_sid",
    delta_load_column="event_time"
)

process_gold_table (
    source_table="silver_sales",
    target_table="gold_fact_sales",
    surrogate_key="transaction_sid",
    delta_load_column="transaction_time",
    timestamp_key="transaction_time",
    dim_table_refs=[
        {"table_name": "gold_dim_product", "join_key": "product_id", "surrogate_key": "product_sid"},
        {"table_name": "gold_dim_customer", "join_key": "customer_id", "surrogate_key": "customer_sid"},
    ]
)

process_gold_table (
    source_table="silver_inventory",
    target_table="gold_fact_inventory",
    surrogate_key="inventory_sid",
    delta_load_column="event_time",
    timestamp_key="event_time",
    dim_table_refs=[{"table_name": "gold_dim_product", "join_key": "product_id", "surrogate_key": "product_sid"}]
)


Processesing changes into gold_dim_product
+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                0|               0|               0|                0|
+-----------------+----------------+----------------+-----------------+


Processesing changes into gold_dim_customer


UnsupportedOperationException: [DELTA_MULTIPLE_SOURCE_ROW_MATCHING_TARGET_ROW_IN_MERGE] Cannot perform Merge as multiple source rows matched and attempted to modify the same
target row in the Delta table in possibly conflicting ways. By SQL semantics of Merge,
when multiple source rows match on the same target row, the result may be ambiguous
as it is unclear which source row should be used to update or delete the matching
target row. You can preprocess the source table to eliminate the possibility of
multiple matches. Please refer to
https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge